In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json, col

In [3]:
spark = SparkSession.builder \
    .appName("Kafka Streaming Example") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .getOrCreate()

# Start Spark with Kafka connector

24/07/26 12:29:30 WARN Utils: Your hostname, Macbook-Air-31.local resolves to a loopback address: 127.0.0.1; using 192.168.68.102 instead (on interface en0)
24/07/26 12:29:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/enriquegomeztagle/.ivy2/cache
The jars for the packages stored in: /Users/enriquegomeztagle/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-96830436-ed36-4ca7-92d0-7e3c1daf3b91;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/enriquegomeztagle/anaconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 172ms :: artifacts dl 7ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central in [default]
	org.apache.kafka#kafka-clients;2.6.0 from central in [default]
	org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 from central in [default]
	org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 from central in [default]
	org.lz4#lz4-java;1.7.1 from central in [default]
	org.slf4j#s

In [4]:
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Gender", StringType(), True)
])

In [5]:
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "test-topic") \
    .option("startingOffsets", "earliest") \
    .load()

# Read data from Kafka

In [6]:
kafka_df_string = kafka_df.selectExpr("CAST(value AS STRING)")
# Convert value from bytes to string

In [7]:
parsed_df = kafka_df_string.select(from_json(col("value"), schema).alias("data")).select("data.*")
# Parse JSON and apply schema

In [8]:
print("Esquema del DataFrame en streaming:")
parsed_df.printSchema()
# Show DF Schema on streaming

Esquema del DataFrame en streaming:
root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)



In [ ]:
query = parsed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()
# View Straming DF on Console

query.awaitTermination()
# Keep Execution

24/07/26 12:29:32 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/73/m50h3qv91_n45vnh3sfw1pb00000gq/T/temporary-4f73f36c-f966-4a6d-b4d2-42812a7b0257. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/26 12:29:32 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/07/26 12:29:32 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/07/26 12:29:32 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/07/26 12:29:32 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/07/26 12:29:32 WARN AdminClientConfig: The configuration '

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+----+------+
|   Name| Age|Gender|
+-------+----+------+
|   NULL|NULL|  NULL|
|  Alice|  25|     F|
|    Bob|  30|     M|
|Charlie|  28|     M|
|ChDavid|  30|     M|
|    Bob|  30|     M|
|   Lion|  80|     A|
|   Lion|  80|     P|
|   Lion|  80|     W|
+-------+----+------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+---+------+
|Name|Age|Gender|
+----+---+------+
| Bob| 30|    M2|
+----+---+------+



24/07/26 12:30:05 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 0 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
24/07/26 12:30:05 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 0 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
24/07/26 12:30:05 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 0 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
24/07/26 12:30:06 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 0 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
24/07/26 12:30:07 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 0 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
